In [0]:
spark.version

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS taxi_dev.bronze;

CREATE VOLUME IF NOT EXISTS taxi_dev.bronze.nyc_taxi;

CREATE SCHEMA IF NOT EXISTS taxi_dev.raw;

CREATE VOLUME IF NOT EXISTS taxi_dev.raw.nyc_taxi;



In [0]:
%sql

SHOW VOLUMES IN taxi_dev.bronze;


In [0]:
dbutils.fs.ls("/Volumes/taxi_dev/raw/nyc_taxi/yellow/year=2025/month=01")


In [0]:
dbutils.fs.ls("/Volumes/taxi_dev/raw/nyc_taxi/yellow/year=2025/month=01")


In [0]:
raw_base_path = "/Volumes/taxi_dev/raw/nyc_taxi/yellow"

df_raw = (
    spark.read
    .format("parquet")
    .load(raw_base_path)
)

df_raw.printSchema()


In [0]:
df_raw.select("year", "month").distinct().show()


In [0]:
from pyspark.sql.functions import current_timestamp, col

df_bronze = (
    df_raw
    .withColumn("ingestion_ts", current_timestamp())
    .withColumn("source_file", col("_metadata.file_path"))
)


In [0]:
bronze_path = "/Volumes/taxi_dev/bronze/nyc_taxi/yellow_taxi"

(
    df_bronze
    .write
    .format("delta")
    .mode("append")
    .partitionBy("year", "month")
    .save(bronze_path)
)


In [0]:
%sql
SELECT year, month, COUNT(*) 
FROM taxi_dev.bronze.yellow_taxi_trips
GROUP BY year, month;
